In [1]:
import numpy as np

from src.estimator.scalar_mean import ScalarMeanClient
from src.server import Server

In [27]:
N = 5
clients = []
server = Server(lambda x: x)

mus = np.random.normal(10, 2, (N, 1))
num_samples = np.random.randint(100, 1000, N)
local_estimations = np.zeros((N, 1))
local_variances = np.zeros((N, 1))

for i in range(N):
    clients.append(ScalarMeanClient(i, num_samples[i], mus[i].reshape(1, 1), lambda x: 1 / (1 + x)))
    local_estimations[i, :] = clients[i].local_estimate()
    local_variances[i, :] = clients[i].bootstrap_variance()
    
aggregate = server.aggregate(local_estimations, local_variances)

for i in range(N):
    print(clients[i].id, clients[i].gain(aggregate[i]))

0 0.0005070409778935581
1 -0.054745978916880156
2 -0.03466350224146775
3 0.008566102659277597
4 -0.03904473430791067


In [34]:
1 / (1 + (aggregate - mus) ** 2), 1 / (1 + (local_estimations - mus) ** 2)

(array([[0.98071627],
        [0.94317707],
        [0.96516603],
        [0.99659505],
        [0.96046289]]),
 array([[0.98020923],
        [0.99792305],
        [0.99982953],
        [0.98802895],
        [0.99950762]]))

In [10]:
aggregate

array([[15.06442216],
       [13.98179496],
       [14.86232809],
       [10.6396438 ],
       [ 3.65230372]])

In [3]:
B = local_estimations
variances = local_variances

In [4]:
local_estimations, mus

(array([[15.06567954],
        [13.98296198],
        [14.8635686 ],
        [10.64053186],
        [ 3.65260857]]),
 array([14.96320396, 13.98523772, 14.83009385, 10.63699442,  3.66056572]))

In [5]:
N = B.shape[0]
V = np.eye(N) * variances.squeeze(axis=1)
    
print(B)
C = B @ B.T
K = B @ B.T
W = K @ np.linalg.inv(C + V)
B = (np.expand_dims(B, axis=0).repeat(repeats=N, axis=0) * np.expand_dims(W, axis=-1)).sum(axis=1)

[[15.06567954]
 [13.98296198]
 [14.8635686 ]
 [10.64053186]
 [ 3.65260857]]


In [6]:
W

array([[9.22396649e-04, 1.05999776e+00, 1.06695647e-03, 1.96087418e-02,
        1.12550205e-03],
       [8.56107236e-04, 9.83819439e-01, 9.90278054e-04, 1.81995303e-02,
        1.04461616e-03],
       [9.10022401e-04, 1.04577755e+00, 1.05264291e-03, 1.93456842e-02,
        1.11040308e-03],
       [6.51466859e-04, 7.48651258e-01, 7.53566033e-04, 1.38491889e-02,
        7.94915381e-04],
       [2.23631061e-04, 2.56991853e-01, 2.58678963e-04, 4.75405427e-03,
        2.72873083e-04]])

In [7]:
clients[0].var ** 2, clients[0].bootstrap_variance(), clients[0].num_samples

(20.643424640457905, 20.746978415330904, 4404)

In [8]:
clients[0].local_estimate(), clients[0].ground_truth

(array([[15.06567954]]), array([[14.96320396]]))

In [9]:
clients[0].mse(clients[0].local_estimate())

0.01050124415732399